<a href="https://colab.research.google.com/github/engineerinvestor/Portfolio-Analysis/blob/main/Interactive_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive Portfolio Analysis

An interactive widget-based interface for analyzing investment portfolios.

**Features:**
- Preset portfolios (60/40, Three-Fund, All-Weather, etc.)
- Custom portfolio builder
- Date range selection
- Monte Carlo simulation with adjustable parameters
- Benchmark comparison

Works in Google Colab with `ipywidgets`.

In [ ]:
!pip install pandas yfinance numpy matplotlib ipywidgets

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output

## Preset Portfolios

Common portfolio allocations for quick analysis.

In [ ]:
PRESET_PORTFOLIOS = {
    'Custom': {},
    '60/40 Traditional': {
        'VTI': 0.60,   # Total US Stock Market
        'BND': 0.40    # Total US Bond Market
    },
    'Three-Fund Portfolio': {
        'VTI': 0.40,   # Total US Stock Market
        'VXUS': 0.20,  # Total International Stock
        'BND': 0.40    # Total US Bond Market
    },
    'All-Weather (Ray Dalio)': {
        'VTI': 0.30,   # Stocks
        'TLT': 0.40,   # Long-term Treasury Bonds
        'IEF': 0.15,   # Intermediate Treasury Bonds
        'GLD': 0.075,  # Gold
        'DBC': 0.075   # Commodities
    },
    'Golden Butterfly': {
        'VTI': 0.20,   # Total Stock Market
        'VBR': 0.20,   # Small Cap Value
        'TLT': 0.20,   # Long-term Treasury
        'SHY': 0.20,   # Short-term Treasury
        'GLD': 0.20    # Gold
    },
    'Aggressive Growth': {
        'VTI': 0.50,   # Total US Stock Market
        'VGT': 0.25,   # Technology
        'VXUS': 0.25   # International
    },
    'Conservative Income': {
        'VTI': 0.20,   # Total US Stock Market
        'BND': 0.50,   # Total Bond Market
        'VTIP': 0.15,  # TIPS
        'VNQ': 0.15    # Real Estate
    },
    'S&P 500 Only': {
        'SPY': 1.0
    }
}

## Core Analysis Classes

Same classes from Basic_Portfolio_Analysis.ipynb for standalone functionality.

In [ ]:
class DataLoader:
    def __init__(self, tickers, start_date, end_date):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date

    def fetch_data(self):
        data = yf.download(self.tickers, start=self.start_date, end=self.end_date, progress=False)['Adj Close']
        return data


class PerformanceMetrics:
    @staticmethod
    def calculate_annual_return(data):
        annual_return = data.resample('Y').last().pct_change().mean()
        return annual_return

    @staticmethod
    def calculate_annual_volatility(data):
        annual_volatility = data.pct_change().std() * (252 ** 0.5)
        return annual_volatility

    @staticmethod
    def calculate_sharpe_ratio(data, risk_free_rate=0.02):
        annual_return = PerformanceMetrics.calculate_annual_return(data)
        annual_volatility = PerformanceMetrics.calculate_annual_volatility(data)
        sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility
        return sharpe_ratio

    @staticmethod
    def calculate_max_drawdown(data):
        cumulative_returns = (1 + data.pct_change()).cumprod()
        peak = cumulative_returns.expanding(min_periods=1).max()
        drawdown = (cumulative_returns / peak) - 1
        max_drawdown = drawdown.min()
        return max_drawdown


class PortfolioAnalysis:
    def __init__(self, data, weights):
        self.data = data
        self.weights = np.array(weights)

    def calculate_portfolio_return(self):
        returns = self.data.pct_change().mean()
        portfolio_return = np.dot(self.weights, returns) * 252
        return portfolio_return

    def calculate_portfolio_volatility(self):
        returns = self.data.pct_change()
        covariance_matrix = returns.cov() * 252
        portfolio_volatility = np.sqrt(np.dot(self.weights.T, np.dot(covariance_matrix, self.weights)))
        return portfolio_volatility

    def calculate_portfolio_sharpe_ratio(self, risk_free_rate=0.02):
        portfolio_return = self.calculate_portfolio_return()
        portfolio_volatility = self.calculate_portfolio_volatility()
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        return sharpe_ratio


class MonteCarloSimulation:
    def __init__(self, data, weights, num_simulations=1000, time_horizon=252, initial_investment=10000):
        self.data = data
        self.weights = np.array(weights)
        self.num_simulations = num_simulations
        self.time_horizon = time_horizon
        self.initial_investment = initial_investment
        self._results = None

    def simulate(self):
        returns = self.data.pct_change().dropna()
        mean_returns = returns.mean().values
        cov_matrix = returns.cov().values
        results = np.zeros((self.num_simulations, self.time_horizon))
        
        for i in range(self.num_simulations):
            sim_returns = np.random.multivariate_normal(mean_returns, cov_matrix, self.time_horizon)
            portfolio_returns = sim_returns @ self.weights
            cumulative_returns = np.cumprod(1 + portfolio_returns)
            results[i, :] = self.initial_investment * cumulative_returns
        
        self._results = results
        return results

    def get_statistics(self):
        if self._results is None:
            self.simulate()
        results = self._results
        final_values = results[:, -1]
        return {
            'mean': np.mean(final_values),
            'median': np.median(final_values),
            'percentile_5': np.percentile(final_values, 5),
            'percentile_95': np.percentile(final_values, 95),
            'prob_loss': np.mean(final_values < self.initial_investment) * 100
        }

    def plot_simulation(self, ax=None):
        if self._results is None:
            self.simulate()
        results = self._results
        
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 6))
        
        for i in range(min(100, self.num_simulations)):
            ax.plot(results[i, :], color='lightblue', alpha=0.3, linewidth=0.5)
        
        days = np.arange(self.time_horizon)
        p5 = np.percentile(results, 5, axis=0)
        p50 = np.percentile(results, 50, axis=0)
        p95 = np.percentile(results, 95, axis=0)
        
        ax.fill_between(days, p5, p95, color='blue', alpha=0.2, label='5th-95th percentile')
        ax.plot(p50, color='darkblue', linewidth=2, label='Median')
        ax.axhline(y=self.initial_investment, color='red', linestyle='--', label=f'Initial: ${self.initial_investment:,.0f}')
        
        ax.set_title(f'Monte Carlo Simulation ({self.num_simulations:,} paths)')
        ax.set_xlabel('Trading Days')
        ax.set_ylabel('Portfolio Value ($)')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        return ax

## Interactive Portfolio Analyzer

Widget-based interface for portfolio analysis.

In [ ]:
class InteractivePortfolioAnalyzer:
    """
    Interactive widget-based portfolio analyzer for Jupyter/Colab.
    """
    
    def __init__(self):
        self.data = None
        self.tickers = []
        self.weights = []
        self._setup_widgets()
    
    def _setup_widgets(self):
        # Portfolio Selection
        self.preset_dropdown = widgets.Dropdown(
            options=list(PRESET_PORTFOLIOS.keys()),
            value='Three-Fund Portfolio',
            description='Preset:',
            style={'description_width': '80px'}
        )
        self.preset_dropdown.observe(self._on_preset_change, names='value')
        
        # Custom portfolio inputs
        self.custom_tickers = widgets.Text(
            value='VTI, VXUS, BND',
            description='Tickers:',
            placeholder='e.g., VTI, VXUS, BND',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='400px')
        )
        
        self.custom_weights = widgets.Text(
            value='0.4, 0.2, 0.4',
            description='Weights:',
            placeholder='e.g., 0.4, 0.2, 0.4',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='400px')
        )
        
        # Date range
        self.start_date = widgets.DatePicker(
            description='Start:',
            value=datetime.now() - timedelta(days=5*365),
            style={'description_width': '80px'}
        )
        
        self.end_date = widgets.DatePicker(
            description='End:',
            value=datetime.now(),
            style={'description_width': '80px'}
        )
        
        # Analysis options
        self.show_performance = widgets.Checkbox(
            value=True, 
            description='Performance Metrics',
            layout=widgets.Layout(width='200px')
        )
        
        self.show_cumulative = widgets.Checkbox(
            value=True, 
            description='Cumulative Returns',
            layout=widgets.Layout(width='200px')
        )
        
        self.show_monte_carlo = widgets.Checkbox(
            value=True, 
            description='Monte Carlo Simulation',
            layout=widgets.Layout(width='200px')
        )
        
        # Monte Carlo parameters
        self.mc_simulations = widgets.IntSlider(
            value=1000,
            min=100,
            max=5000,
            step=100,
            description='Simulations:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='400px')
        )
        
        self.mc_horizon = widgets.IntSlider(
            value=252,
            min=21,
            max=1260,
            step=21,
            description='Days Forward:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='400px')
        )
        
        self.mc_initial = widgets.IntText(
            value=10000,
            description='Initial ($):',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='200px')
        )
        
        # Risk-free rate
        self.risk_free_rate = widgets.FloatSlider(
            value=0.04,
            min=0.0,
            max=0.10,
            step=0.005,
            description='Risk-Free Rate:',
            readout_format='.1%',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )
        
        # Analyze button
        self.analyze_button = widgets.Button(
            description='Analyze Portfolio',
            button_style='primary',
            icon='chart-line',
            layout=widgets.Layout(width='200px', height='40px')
        )
        self.analyze_button.on_click(self._on_analyze)
        
        # Output area
        self.output = widgets.Output()
        
        # Initialize with preset
        self._on_preset_change({'new': 'Three-Fund Portfolio'})
    
    def _on_preset_change(self, change):
        preset_name = change['new']
        if preset_name != 'Custom' and preset_name in PRESET_PORTFOLIOS:
            portfolio = PRESET_PORTFOLIOS[preset_name]
            self.custom_tickers.value = ', '.join(portfolio.keys())
            self.custom_weights.value = ', '.join([str(w) for w in portfolio.values()])
    
    def _parse_portfolio(self):
        tickers = [t.strip().upper() for t in self.custom_tickers.value.split(',')]
        weights = [float(w.strip()) for w in self.custom_weights.value.split(',')]
        
        if len(tickers) != len(weights):
            raise ValueError(f"Number of tickers ({len(tickers)}) must match weights ({len(weights)})")
        
        weight_sum = sum(weights)
        if not np.isclose(weight_sum, 1.0):
            raise ValueError(f"Weights must sum to 1.0 (currently {weight_sum:.2f})")
        
        return tickers, weights
    
    def _on_analyze(self, button):
        with self.output:
            clear_output(wait=True)
            
            try:
                # Parse portfolio
                self.tickers, self.weights = self._parse_portfolio()
                
                print(f"Analyzing portfolio: {dict(zip(self.tickers, self.weights))}")
                print(f"Date range: {self.start_date.value} to {self.end_date.value}")
                print("\nFetching data...")
                
                # Fetch data
                loader = DataLoader(
                    self.tickers,
                    self.start_date.value.strftime('%Y-%m-%d'),
                    self.end_date.value.strftime('%Y-%m-%d')
                )
                self.data = loader.fetch_data()
                
                if self.data.empty:
                    print("Error: No data returned. Check ticker symbols and date range.")
                    return
                
                # Handle single ticker case
                if isinstance(self.data, pd.Series):
                    self.data = self.data.to_frame(name=self.tickers[0])
                
                print(f"Data loaded: {len(self.data)} trading days\n")
                
                # Performance Metrics
                if self.show_performance.value:
                    self._display_metrics()
                
                # Plots
                num_plots = sum([self.show_cumulative.value, self.show_monte_carlo.value])
                if num_plots > 0:
                    fig, axes = plt.subplots(1, num_plots, figsize=(7*num_plots, 5))
                    if num_plots == 1:
                        axes = [axes]
                    
                    plot_idx = 0
                    
                    if self.show_cumulative.value:
                        self._plot_cumulative(axes[plot_idx])
                        plot_idx += 1
                    
                    if self.show_monte_carlo.value:
                        self._plot_monte_carlo(axes[plot_idx])
                        plot_idx += 1
                    
                    plt.tight_layout()
                    plt.show()
                    
            except Exception as e:
                print(f"Error: {str(e)}")
                import traceback
                traceback.print_exc()
    
    def _display_metrics(self):
        portfolio = PortfolioAnalysis(self.data, self.weights)
        
        port_return = portfolio.calculate_portfolio_return()
        port_vol = portfolio.calculate_portfolio_volatility()
        port_sharpe = portfolio.calculate_portfolio_sharpe_ratio(self.risk_free_rate.value)
        
        # Calculate portfolio max drawdown
        returns = self.data.pct_change().dropna()
        weighted_returns = returns.dot(self.weights)
        cum_returns = (1 + weighted_returns).cumprod()
        peak = cum_returns.expanding(min_periods=1).max()
        drawdown = (cum_returns / peak) - 1
        max_dd = drawdown.min()
        
        print("="*50)
        print("PORTFOLIO PERFORMANCE METRICS")
        print("="*50)
        print(f"Annual Return:      {port_return*100:>10.2f}%")
        print(f"Annual Volatility:  {port_vol*100:>10.2f}%")
        print(f"Sharpe Ratio:       {port_sharpe:>10.2f}")
        print(f"Max Drawdown:       {max_dd*100:>10.2f}%")
        print(f"Risk-Free Rate:     {self.risk_free_rate.value*100:>10.2f}%")
        print("="*50 + "\n")
    
    def _plot_cumulative(self, ax):
        returns = self.data.pct_change().dropna()
        weighted_returns = returns.dot(self.weights)
        cumulative = (1 + weighted_returns).cumprod()
        
        ax.plot(cumulative.index, cumulative.values, linewidth=2, color='blue')
        ax.set_title('Portfolio Cumulative Returns')
        ax.set_xlabel('Date')
        ax.set_ylabel('Growth of $1')
        ax.grid(True, alpha=0.3)
        
        # Add final value annotation
        final_val = cumulative.iloc[-1]
        ax.annotate(f'${final_val:.2f}', 
                   xy=(cumulative.index[-1], final_val),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=10, color='blue')
    
    def _plot_monte_carlo(self, ax):
        mc = MonteCarloSimulation(
            self.data,
            self.weights,
            num_simulations=self.mc_simulations.value,
            time_horizon=self.mc_horizon.value,
            initial_investment=self.mc_initial.value
        )
        mc.plot_simulation(ax=ax)
        
        # Print MC statistics
        stats = mc.get_statistics()
        print("MONTE CARLO PROJECTION")
        print("-"*40)
        print(f"Initial Investment:  ${self.mc_initial.value:,.0f}")
        print(f"Time Horizon:        {self.mc_horizon.value} days")
        print(f"Median Final Value:  ${stats['median']:,.0f}")
        print(f"5th Percentile:      ${stats['percentile_5']:,.0f}")
        print(f"95th Percentile:     ${stats['percentile_95']:,.0f}")
        print(f"Probability of Loss: {stats['prob_loss']:.1f}%")
        print("-"*40 + "\n")
    
    def display(self):
        """Display the interactive widget interface."""
        # Layout
        portfolio_box = widgets.VBox([
            widgets.HTML('<h3>Portfolio Selection</h3>'),
            self.preset_dropdown,
            self.custom_tickers,
            self.custom_weights
        ])
        
        date_box = widgets.VBox([
            widgets.HTML('<h3>Date Range</h3>'),
            widgets.HBox([self.start_date, self.end_date])
        ])
        
        options_box = widgets.VBox([
            widgets.HTML('<h3>Analysis Options</h3>'),
            widgets.HBox([self.show_performance, self.show_cumulative, self.show_monte_carlo]),
            self.risk_free_rate
        ])
        
        mc_box = widgets.VBox([
            widgets.HTML('<h3>Monte Carlo Parameters</h3>'),
            self.mc_simulations,
            self.mc_horizon,
            self.mc_initial
        ])
        
        left_panel = widgets.VBox([portfolio_box, date_box])
        right_panel = widgets.VBox([options_box, mc_box])
        
        top_panel = widgets.HBox([left_panel, right_panel], 
                                  layout=widgets.Layout(justify_content='space-around'))
        
        display(widgets.VBox([
            widgets.HTML('<h2>Interactive Portfolio Analyzer</h2>'),
            top_panel,
            widgets.HBox([self.analyze_button], layout=widgets.Layout(justify_content='center', margin='20px 0')),
            widgets.HTML('<hr>'),
            self.output
        ]))

## Launch Interactive Analyzer

Run the cell below to start the interactive portfolio analyzer.

In [ ]:
# Create and display the interactive analyzer
analyzer = InteractivePortfolioAnalyzer()
analyzer.display()

## Tips

1. **Preset Portfolios**: Select from common allocations or choose "Custom" to enter your own
2. **Tickers**: Use standard ticker symbols (VTI, SPY, AAPL, etc.)
3. **Weights**: Must sum to 1.0 (e.g., 0.6, 0.4 for 60/40)
4. **Monte Carlo**: Adjust simulations and time horizon to explore different scenarios
5. **Date Range**: Longer history provides more data for analysis but may not include newer ETFs